## Create Data Maps from NYSDEC Permit Applications

In [1]:
from GeoCodes import *
% matplotlib notebook

In [2]:
#---Read in data to begin analysis

#---Permit data
f = r'/home/slawler/Desktop/RiskFactorData/permits/permit_intersect.txt'
data = pd.read_csv(f, sep = ',')

#---Flow data
f_flow = r'/home/slawler/Desktop/RiskFactorData/flows/B_NormalFlows_int.txt'
flow_data = pd.read_csv(f_flow, sep = '\t')

huc12 = r'/home/slawler/Desktop/RiskFactorData/hucs_bridges.txt'
huc12_data = pd.read_csv(huc12, sep = '\t')

In [3]:
#-- Count Permits by type from list:
'''
DA = Dams and Impoundments; 
DO = Docks and Moorings; 
EF = Excavation and Fill in Navigable Waters; 
SD = Stream Disturbance of Protected Stream; 
WQ = Water Quality Certification
'''

p_types = data['DEC_Jurisd'].unique()
p_total = 0 #--Added as a check to make sure the numbers make sense 

for p in p_types:
    permits = (data['DEC_Jurisd']== p).sum()
    #print(p, permits)
    p_total += permits

#print('\nTotal Permits = ', p_total)

In [4]:
#--Select subset of permits: Stream Disturbance only

df_SD = data[data['DEC_Jurisd']== 'SD']

#--Get a count of permits by 'Project' (Batch_Numb field) 
b_types = df_SD['Batch_Numb'].unique() 

print('Stream Disturbance of Protected Stream: ', len(df_SD), 'records', len(b_types),'Projects')

Stream Disturbance of Protected Stream:  20133 records 19108 Projects


In [5]:
#-- Get a count of permits by 'HUC 12'

df_HUC = df_SD.groupby('HUC_12').count()

b_types = df_SD['Batch_Numb'].unique()

#-- Rank the HUC 12's by number of permits 

df_HUC_sorted = df_HUC.sort_values(by = ['Batch_Numb'], ascending=False)

print('Stream Disturbance of Protected Stream: ', len(df_HUC), 'HUCs', len(b_types),'Projects')

Stream Disturbance of Protected Stream:  1235 HUCs 19108 Projects


In [6]:
# Select fields from the Permit dataset to include in a map

usecols = ['x','y','Short_Proj','owner'] #--all columns after x,y used in map's Pop-up

data_popup = usecols[2:len(usecols)] 

data[usecols].head(2)

,x,y,Short_Proj,owner
0,-73.659185,40.990066,Seawall replacement,"PORT CHESTER, VILLAGE OF"
1,-73.659185,40.990066,Seawall replacement,MUTO*SALVATORE


In [7]:
#-- Merge datasets

flow_data[['BIN']] = flow_data[['BIN']].astype(str)
huc12_data[['BIN']] = huc12_data[['BIN']].astype(str)

df_total = pd.merge(flow_data,huc12_data, on = 'BIN', how = 'outer')
df_total.head()
flow_data.head()
huc12_data.head()
#df_total.head(2)

,HUC_12,HU_12_NAME,BIN,CROSSED,y,x
0,20301030204,Middle Ramapo River,3346140,RAMAPO RIVER,41.125047,-74.164490
1,20301030204,Middle Ramapo River,1027590,RAMAPO RIVER,41.132272,-74.167019
2,20301030204,Middle Ramapo River,2346230,RAMAPO RIVER,41.140479,-74.168398
3,11000060403,Byram River,1091459,BYRAM RIVER,41.136756,-73.701354
4,20301030102,Ringwood River,3345050,RINGWOOD RIVER,41.152594,-74.240056


In [8]:
# Select fields from the Permit dataset to include in a map

usecols2 = ['x','y','BIN','Q100', 'HUC_12'] #--all columns after x,y used in map's Pop-up

data_popup2 = usecols2[2:len(usecols2)] 

df_total[usecols2].head(2)

,x,y,BIN,Q100,HUC_12
0,-73.696498,42.704323,1000160,2830.0,2.020006e+10
1,-73.631403,42.732831,1000190,19.0,2.020006e+10


In [9]:
field = 'HUC_12'


mapdir = '/home/slawler/Desktop/RiskFactorData/analysis'
scriptdir = '/home/slawler/Desktop/RiskFactorData/scripts'

item = 20200060905


df = data.loc[data[field] == item]
df2 = df_total.loc[df_total[field] == item]
xbar = df['x'].mean()
ybar = df['y'].mean()
geoj =df_to_geojson(df, data_popup, lat= 'y', lon='x')
geoj2 =df_to_geojson(df2, data_popup2, lat= 'y', lon='x')

newmap = os.path.join(mapdir,'{}_{}.html'.format(field,str(item)))
map_template = os.path.join(scriptdir,'map_template.html')

mapit(newmap,geoj,geoj2,xbar,ybar,map_template)

File Created:  /home/slawler/Desktop/RiskFactorData/analysis/HUC_12_20200060905.html
